### This notebook demonstrates how to evaluate accuracy

In [1]:
from sklearn.neighbors import NearestNeighbors
from TfidfVectorizer import TfidfVectorizer
from tqdm import tqdm
from utils import *

# don't forget to set clean=True to clean the data!
data = load_data_from_json('data/data_5d.json', clean=True, use_examples=True)
data_test = load_data_from_json('data/test_200.json')

100%|██████████| 103874/103874 [00:12<00:00, 8217.83it/s] 


In [2]:
data_train, data_seen_500, data_unseen_500 = split_seen_unseen(data)
print(len(list(itertools.chain(*data_train.values()))))
print(len(list(itertools.chain(*data_seen_500.values()))))
print(len(list(itertools.chain(*data_unseen_500.values()))))

576612
500
500


### Fit training data

In [3]:
# fit tfidf vectorizer
vectorizer = TfidfVectorizer(stop_words=None, stemmer=None, norm=False)
X_tfidf = vectorizer.fit_transform(data_train) # <--- fit train data here
words = vectorizer.words

# fit knn
knn = NearestNeighbors(metric='cosine')
knn.fit(X_tfidf)

100%|██████████| 96213/96213 [00:51<00:00, 1864.02it/s] 


NearestNeighbors(metric='cosine')

### Test seen data

In [4]:
y_pred = []
y_gold = []
for word, defi in tqdm(data_seen_500.items()): # <--- use seen_500 data here
    query = list(defi)[0]
    query = vectorizer.transform(query)

    prediction = search(query, knn, words, n=1000)

    y_pred.append(prediction)
    y_gold.append(word)

evaluate(y_pred, y_gold)

100%|██████████| 36/36 [00:07<00:00,  4.62it/s]

acc@1: 0.64
acc@10: 0.78
acc@100: 0.94
median rank: 0
var rank: 51


(0.6388888888888888,
 0.7777777777777778,
 0.9444444444444444,
 0.0,
 51.58239094454188)

### Test unseen data

In [5]:
y_pred = []
y_gold = []
for word, defi in tqdm(data_unseen_500.items()): # <--- use unseen_500 data here
    query = list(defi)[0]
    query = vectorizer.transform(query)

    prediction = search(query, knn, words, n=1000)

    y_pred.append(prediction)
    y_gold.append(word)

evaluate(y_pred, y_gold)

100%|██████████| 485/485 [01:29<00:00,  5.42it/s]

acc@1: 0.1
acc@10: 0.24
acc@100: 0.47
median rank: 140
var rank: 422


(0.09896907216494845,
 0.23505154639175257,
 0.465979381443299,
 140.0,
 422.97072003811957)

### Test description set (200) data

In [6]:
# fit tfidf vectorizer
vectorizer = TfidfVectorizer(stop_words=None, stemmer=None, norm=False)
X_tfidf = vectorizer.fit_transform(data) # <--- fit all data here
words = vectorizer.words

# fit knn
knn = NearestNeighbors(metric='cosine')
knn.fit(X_tfidf)

y_pred = []
y_gold = []
for word, defi in tqdm(data_test.items()): # <--- use desc_200 data here
    query = list(defi)[0]
    query = vectorizer.transform(query)

    prediction = search(query, knn, words, n=1000)

    y_pred.append(prediction)
    y_gold.append(word)

evaluate(y_pred, y_gold)

100%|██████████| 200/200 [00:34<00:00,  5.72it/s]

acc@1: 0.02
acc@10: 0.06
acc@100: 0.28
median rank: 316
var rank: 404


(0.02, 0.06, 0.285, 316.5, 404.6999031381155)